In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import random
import os
import requests
from bs4 import BeautifulSoup, Comment
from datetime import datetime
import html5lib
import lxml

from utils import mp_to_minutes

In [2]:
# extract draft class from the url
def load_draft_class(draft_year):
    url = f"https://www.basketball-reference.com/draft/NBA_{draft_year}.html"

    df = pd.read_html(url)[0]
    df_links = pd.read_html(url, extract_links="body")[0]

    df.columns = [f"{a}_{b}".strip("_") for a, b in df.columns]
    df_links.columns = df.columns

    df = df[df["Round 1_Player"].notna()].copy()

    df["player"] = df["Round 1_Player"]
    df["player_id"] = df_links["Round 1_Player"].apply(
        lambda x: x[1].split("/")[-1].replace(".html", "")
        if isinstance(x, tuple) and x[1]
        else None
    )

    # 🔑 drop header rows like "Round 2"
    df = df[df["player_id"].notna()].copy()

    df["draft_year"] = draft_year
    df["rookie_season"] = draft_year + 1

    return df[["draft_year", "player", "player_id", "rookie_season"]]



In [3]:
# load or scrape draft class with caching
# def load_or_scrape_draft(year):
#     path = f"assets/drafts/draft_{year}.csv"
#     if os.path.exists(path):
#         return pd.read_csv(path)
    
#     df = load_draft_class(year)
#     df.to_csv(path, index=False)
#     return df

# function to scrape a single draft class
def scrape_and_save_year(year):
    df = load_draft_class(year)
    df.to_csv(f"assets/drafts/draft_{year}.csv", index=False)
    print(f"✓ saved {year}")


In [3]:
# freeze the draft years collected
import glob
draft_classes = pd.concat(
    [pd.read_csv(f) for f in glob.glob("assets/drafts/draft_20*.csv")],
    ignore_index=True
)
draft_classes.head(100)
draft_classes.to_csv("assets/draft_classes.csv", index=False)


In [4]:
# override rookie seasons for specific players
rookie_overrides = {
    "griffbl01": 2011,  # drafted 2009, rookie season 2010–11
    "embiijo01": 2016,  # drafted 2014, rookie season 2015-16
}

In [5]:
def load_rookie_gamelog(player_id, season):
    first_letter = player_id[0]
    url = (
        f"https://www.basketball-reference.com/players/"
        f"{first_letter}/{player_id}/gamelog/{season}"
    )

    df = pd.read_html(
        url,
        attrs={"id": "player_game_log_reg"}
    )[0]

    # 1️⃣ KEEP ONLY REAL GAMES (must have an opponent)
    df = df[df["Opp"].notna()].copy()

    # 2️⃣ Convert MP to numeric (DNPs become NaN)
    df["MP"] = df["MP"].apply(mp_to_minutes)

    # 3️⃣ Drop games not played
    df = df[df["MP"].notna()].copy()

    return df


In [6]:
def aggregate_capped_minutes(
    gamelog,
    cap_minutes=900,
    stat_cols=("PTS", "TRB", "AST", "STL", "BLK", "TOV"),
):
    """
    Aggregate rookie production up to a capped number of minutes.
    """

    # Ensure counting stats are numeric
    gamelog = gamelog.copy()
    for stat in stat_cols:
        gamelog[stat] = pd.to_numeric(gamelog[stat], errors="coerce")

    total_minutes = 0.0
    totals = {stat: 0.0 for stat in stat_cols}

    for _, row in gamelog.iterrows():
        mp = row["MP"]

        if total_minutes >= cap_minutes:
            break

        remaining = cap_minutes - total_minutes

        if mp <= remaining:
            weight = 1.0
            used_mp = mp
        else:
            weight = remaining / mp
            used_mp = remaining

        total_minutes += used_mp

        for stat in stat_cols:
            if pd.notna(row[stat]):
                totals[stat] += row[stat] * weight

    return {
        "minutes_used": total_minutes,
        **totals,
    }

In [7]:
# calculate rate stats
def add_rate_stats(agg, minutes_base=36):
    factor = minutes_base / agg["minutes_used"]
    return {
        f"{k}_per_{minutes_base}": v * factor
        for k, v in agg.items()
        if k not in ("minutes_used",)
    }

In [8]:
rookie_caps_900 = pd.read_csv("assets/rookie_caps_900.csv")
errors_df = pd.read_csv("assets/rookie_errors.csv")
draft_classes = pd.read_csv("assets/draft_classes.csv")

In [9]:
done_keys = set(zip(rookie_caps_900.player_id, rookie_caps_900.rookie_season))

In [10]:
failed_keys = set(
    zip(
        errors_df.loc[~errors_df["error"].str.contains("429", na=False), "player_id"],
        errors_df.loc[~errors_df["error"].str.contains("429", na=False), "rookie_season"],
    )
)


In [11]:
draft_classes["_key"] = list(zip(draft_classes.player_id, draft_classes.rookie_season))

draft_classes["_done"] = draft_classes["_key"].isin(done_keys)
draft_classes["_failed"] = draft_classes["_key"].isin(failed_keys)

todo = draft_classes[~draft_classes["_done"] & ~draft_classes["_failed"]].copy()

len(todo), todo.head()

(1231,
     draft_year           player  player_id  rookie_season               _key  \
 19        2000   Speedy Claxton  claxtsp01           2001  (claxtsp01, 2001)   
 26        2000    Primož Brezec  brezepr01           2001  (brezepr01, 2001)   
 29        2000      Marko Jarić  jaricma01           2001  (jaricma01, 2001)   
 40        2000  Chris Carrawell  carrach01           2001  (carrach01, 2001)   
 45        2000  DeeAndre Hulett  huletde01           2001  (huletde01, 2001)   
 
     _done  _failed  
 19  False    False  
 26  False    False  
 29  False    False  
 40  False    False  
 45  False    False  )

In [12]:
new_results = []
new_errors = []
CHECKPOINT_EVERY = 25

for i, (_, row) in enumerate(todo.iterrows(), 1):
    try:
        gamelog = load_rookie_gamelog(row.player_id, row.rookie_season)
        agg = aggregate_capped_minutes(gamelog, cap_minutes=900)

        new_results.append({**row.to_dict(), **agg})

        if i % CHECKPOINT_EVERY == 0:
            print(f"Processed {i}/{len(todo)} — checkpointing")

            rookie_caps_900 = (
                pd.concat([rookie_caps_900, pd.DataFrame(new_results)], ignore_index=True)
                .sort_values("minutes_used", ascending=False)
                .drop_duplicates(subset=["player_id", "rookie_season"], keep="first")
                .reset_index(drop=True)
            )

            rookie_caps_900.to_csv("assets/rookie_caps_900.csv", index=False)

            new_results = []  # clear buffer

        time.sleep(random.uniform(12, 18))

    except Exception as e:
        new_errors.append({
            "player_id": row.player_id,
            "rookie_season": row.rookie_season,
            "error": str(e),
        })


In [13]:
# Flush in-memory progress safely

if new_results:
    rookie_caps_900 = (
        pd.concat([rookie_caps_900, pd.DataFrame(new_results)], ignore_index=True)
        .sort_values("minutes_used", ascending=False)
        .drop_duplicates(subset=["player_id", "rookie_season"], keep="first")
        .reset_index(drop=True)
    )

    rookie_caps_900.to_csv("assets/rookie_caps_900.csv", index=False)
    print(f"Saved {len(new_results)} new rows to rookie_caps_900")

    new_results = []

if new_errors:
    errors_df = (
        pd.concat([errors_df, pd.DataFrame(new_errors)], ignore_index=True)
        .drop_duplicates(subset=["player_id", "rookie_season"])
        .reset_index(drop=True)
    )

    errors_df.to_csv("assets/rookie_errors.csv", index=False)
    print(f"Saved {len(new_errors)} new errors")

    new_errors = []


Saved 1231 new errors


In [27]:
existing = set(zip(rookie_caps_900.player_id, rookie_caps_900.rookie_season))
new_keys = [(r["player_id"], r["rookie_season"]) for r in new_results]

sum(k in existing for k in new_keys), len(new_keys)

(30, 30)

In [54]:
# new completed and failed key sets
done_keys = set(zip(rookie_caps_900.player_id, rookie_caps_900.rookie_season))
failed_keys = set(zip(errors_df.player_id, errors_df.rookie_season))

In [55]:
list(done_keys)[:5], list(failed_keys)[:5]


([('nachbbo01', 2003),
  ('satteke01', 2002),
  ('salmojo01', 2003),
  ('johnsde03', 2001),
  ('turkohe01', 2001)],
 [('thomaet01', 2001)])

In [56]:
("flaggco01", 2026) in done_keys, ("flaggco01", 2026) in failed_keys


(False, False)

In [57]:
# rebuild todo dataframe safely
draft_classes["_key"] = list(zip(draft_classes.player_id, draft_classes.rookie_season))

draft_classes["_done"] = draft_classes["_key"].isin(done_keys)
draft_classes["_failed"] = draft_classes["_key"].isin(failed_keys)

todo = draft_classes[~draft_classes["_done"] & ~draft_classes["_failed"]].copy()

len(todo), todo.head()


(1414,
     draft_year           player  player_id  rookie_season               _key  \
 19        2000   Speedy Claxton  claxtsp01           2001  (claxtsp01, 2001)   
 26        2000    Primož Brezec  brezepr01           2001  (brezepr01, 2001)   
 29        2000      Marko Jarić  jaricma01           2001  (jaricma01, 2001)   
 40        2000  Chris Carrawell  carrach01           2001  (carrach01, 2001)   
 45        2000  DeeAndre Hulett  huletde01           2001  (huletde01, 2001)   
 
     _done  _failed  
 19  False    False  
 26  False    False  
 29  False    False  
 40  False    False  
 45  False    False  )